# Phase transition of Shastry-Sutherland lattice

In previous presentations, we have presented the Shastry-Sutherland lattice and its phase diagram, obtained through both classical numerical method and QuEra's Aquila device. The purpose of this notebook is to reproduce some of the results. We will demonstrate

a) Analog Hamiltonian simulation with Rydberg atoms

b) Simulation of Shastry-Sutherland lattice with Amazon Braket Hybrid Jobs

c) Simulation of Shastry-Sutherland lattice with QuEra's Aquila device

## Analog Hamiltonian simulation with Rydberg atoms

Analog Hamiltonian Simulation (AHS) is a quantum computing paradigm that uses a well-controlled quantum system and tunes its parameters to mimic the dynamics of another quantum system, the one we aim to study. An AHS program is a sequence of time-dependent Hamiltonians that govern the quantum dynamics, which can be understood as a unitary acting simultaneously on all qubits. In order to run AHS program with Rydberg atoms, we will first have to understand what type of Hamiltonian can Rydberg atoms simulate.

### Introduction to Rydberg Hamiltonian

Depending on the atomic states we use for the Rydberg system, its Hamiltonian could take different forms. Here we shall focus on the following type of Hamiltonian

\begin{align}
H(t) = \sum_{j=1}^{N-1}\sum_{k = j+1}^N H_{\text{vdW}, j, k} + \sum_{k=1}^N H_{\text{drive}, k}(t),
\end{align}

where $j, k=1,2,\ldots N$ index the atoms (qubits) in the program register. We describe the nature and effect of each term in the Hamiltonian in the following sections, using an array with two atoms as an example.

### Rydberg interaction and quantum register

The first term in $H(t)$ is the van der Waals interaction between all pairs of Rydberg atoms,

\begin{align}
H_{\text{vdW}, j, k} =V_{j,k} \,n_j\, n_k = \frac{C_6}{R_{j,k}^6} \,n_j\, n_k,
\end{align}

where $C_6$ is a fixed interaction coefficient, and $R_{j,k}=|{\bf x}_j-{\bf x}_k|$ is the distance between atoms $j$ and $k$. Here $n_k = |r_k\rangle\langle r_k|$ is the number operator of atom $k$, and we will use $|g_k\rangle$ and $|r_k\rangle$ to denote the ground and Rydberg states of the $k$-th atom, respectively. While the overall coefficient, $C_6$ is a fixed value (determined by the nature of the Rydberg states), the strength of this interaction can be tuned by adjusting the pairwise distance $R_{j,k}$ between atoms. 

To define the van der Waals interaction in an AHS program, we define a quantum **register** which is a two-dimensional layout of neutral atoms.

In [ ]:
from braket.ahs.atom_arrangement import AtomArrangement

register = AtomArrangement()

# Two atoms with separation 4 micrometers
register.add([0, 0])
register.add([0.0, 4e-6]) ;

Note that the coordinates of the atoms are specified with SI units. The atom arrangement can be visualized in the following way

In [ ]:
from ahs_utils import show_register
show_register(register)

### Driving field

The second term of the Hamiltonian represents the effect of a **driving field** that addresses the atoms simultaneously and uniformly

\begin{align}
H_{\text{drive}, k}(t) = \frac{\Omega(t)}{2}\left(e^{i\phi(t)}|g_k\rangle\langle r_k| + e^{-i\phi(t)}|r_k\rangle\langle g_k|\right) - \Delta_\text{global}(t)n_k,
\end{align}

where $\Omega$, $\phi$, and $\Delta_\text{global}$ to denote the **amplitude** (Rabi frequency), laser **phase**, and the **detuning** of the driving laser field. The $\Omega$ part of the driving term is identical to a uniform (time-dependent) _transverse_ magnetic field, whereas the $\Delta_\text{global}$ part implements the effect of a _longitudinal_ magnetic field, in a spin-model representation.


For the purpose of this example, we choose a trapezoidal waveform for $\Omega(t)$ such that $\int_{0}^T\Omega(t)dt=\frac{\pi}{\sqrt{2}}$ where $T$ is the duration of the program. We also set phase and global detuning to zero. As we will explain below, the defined driving field prepares a Bell state on the two Rydberg atoms.



In [ ]:
from braket.timings.time_series import TimeSeries
from braket.ahs.driving_field import DrivingField
import numpy as np

amplitude = TimeSeries.trapezoidal_signal(
    area=np.pi/np.sqrt(2), # rad 
    value_max = 2.5e6, # rad / seconds
    slew_rate_max = 4e14 # rad / seconds^2
)

phase = TimeSeries.constant_like(amplitude, 0.0)
detuning = TimeSeries.constant_like(amplitude, 0.0)

drive = DrivingField(
    amplitude=amplitude,
    phase=phase,
    detuning=detuning
)

Note that the time series of the driving field are specified with SI units. The driving field can be visualized in the following way

In [ ]:
from ahs_utils import show_global_drive
show_global_drive(drive)

### AHS program

With the quantum register and driving field defined, we can simply assemble them together to define an AHS **program**.

In [ ]:
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation
ahs_program = AnalogHamiltonianSimulation(
    register=register,
    hamiltonian=drive
)

The fully-specified program can be inspected with the following command.

In [ ]:
ahs_program.to_ir().dict()

### Rydberg blockade and Bell state

One important characteristic of Rydberg atoms is the Rydberg blockade phenomenon. To understand that, we note that the $C_6$ interaction coefficient takes the value
\begin{align}
C_6 = 5.42\times 10^{-24} ~\text{rad}~ \text{m}^6/\text{s}
\end{align}

for $|r\rangle = |70S_{1/2}\rangle$ of the $^{87}$Rb atoms. For the typical scenario, where atoms are separated by $4\times10^{-6}$ meters, the van der Waals interaction strength is $V_{jk}=1.32\times10^9 \text{rad}/\text{s}$, which is much larger than the typical scale of the Rabi frequency (around $6\times10^6 \text{rad}/\text{s}$). As a result, when the separation of two atoms is within certain distance, it is nearly impossible to drive them to the Rydberg state simultaneously. 

This is called the Rydberg blockade phenomena, illustrated in the figure below (Source: [Browaeys and Lahaye](https://arxiv.org/abs/2002.07413)), where $R$ is the separation between the atoms, and $E$ indicates the energies or frequencies of the different two-atom states as $R$ changes. The vertical arrows indicate the effect of a uniform driving field (with Rabi frequency $\Omega$) that successfully transitions the atoms from the $|gg\rangle$ ground state to the 1-atom excited state $|\psi_+\rangle = (|gr\rangle + |rg\rangle)/\sqrt{2}$ (independent of $R$), but fails to get from there to the doubly-excited state $|rr\rangle$, if $R$ is smaller than $R_b = (C_6 / \Omega)^{1/6}$, the blockade radius.

<u>Note</u>: In the presence of both global Rabi frequency and detuning, it is more accurate to estimate the blockade radius with $R_b = (C_6 / \sqrt{\Omega^2+\Delta^2})^{1/6}$, see [Pichler, et. al.](https://arxiv.org/abs/1808.10816).

<center><img src="Blockade.png" alt="drawing" style="width:250px;"/>

### Running AHS program with local simulator
As explained, the AHS program defined above realizes a Bell state for the pair of atoms due to the Rydberg blockade phenomenon. Below we will run the AHS program on a local simulator to confirm that the program indeed realizes a Bell state.

In [ ]:
from braket.devices import LocalSimulator
simulator = LocalSimulator('braket_ahs')

We can run the AHS program just like running a quantum circuit on other Braket devices. Below we have explicitly specified the values of `steps` and `shots`, which are the number of time steps in the simulation and the number of sampling for the final stats, respectively. One could increase the accuracy of the result by increasing the values of these arguments, at the expense of longer runtime. 

In [ ]:
%%time

result = simulator.run(ahs_program, shots=1000, steps=100).result()
result.get_counts()

The simulation outcome indeed confirms that the program realizes a Bell state for the two Rydberg atoms.

<div class="alert alert-block alert-info">
Exercise 1: Increase the separation between the atoms and observe the increased probability of the $|rr\rangle$ state
</div>


## Simulation of Shastry-Sutherland lattice with Amazon Braket Hybrid Jobs

In this section, we will study the phase transition of the Shastry-Sutherland (SS) lattice with Rydberg atoms. To recap, we show a typical SS lattice below where the interaction between the vertices are labeled as $V_1$, $V_2$ and $V_3$ respectively, and the Hamiltonian is defined as
\begin{align}
H = \frac{\Omega(t)}{2}\sum_i\left(e^{i\phi(t)}|g_i\rangle\langle r_i| + e^{-i\phi(t)}|r_i\rangle\langle g_i|\right) - \Delta(t)\sum_in_i + V_1\sum_{\langle i, j\rangle}n_in_j + V_2\sum_{\langle i, j\rangle}n_in_j + V_3\sum_{\langle i, j\rangle}n_in_j.
\end{align} 

From previous presentations, we have seen that the phases of the SS lattice is determined by two ratios, namely $V_2/V_1$ and $\Delta/V_1$. Depending on the values of these two ratios, the SS lattice is either in the 1/3 or 2/5 phases as shown below (we will not study the other phases in this notebook). One characteristics of these phases is the structure factor, which is calculated
from two-point correlators according to
\begin{align}
S_{\bf q} = \frac{1}{N_s^2}\sum_{i,j}e^{-i{\bf q}\cdot({\bf r}_i-{\bf r}_j)}\langle n_in_j\rangle
\end{align}
where $N_s$ is the number of sites, and ${\bf r}_i$ is the two-dimensional indices of the $i$-th site.

<center><img src="SSlattice.png" alt="drawing" style="width:350px;"/><img src="phases_SS_lattice.png" alt="phases of SS lattice" width="400"/><img src="structure_factors.png" alt="phases of SS lattice" width="800"/>


Here, we are interested in simulating the phase transition of the SS lattice with Rydberg atoms. In particular, we will demonstrate how to use Amazon Braket Hybrid Jobs to parallelize multiple simulation tasks for speeding up the simulation. 


### Register for SS lattice

Similar to the simulation of Bell state, in order to simulate a given SS lattice, we will first need to specify the locations of the atoms that correspond to the vertices in the SS lattice. For convenience, we define a function `get_SS_lattice` that takes in the two ratios $V_2/V_1$ (`V2_over_V1`) and $\Delta/V_1$ (`Delta_over_V1`) and outputs the register for the SS lattice and `filter_atoms`, the indices for the boundary atoms. As we have seen from the presentations, the boundary atoms serve to stabilize the phase (`phase`) we want to realize. Because we will simulate the SS lattice on the Aquila lattice, the lattice need to satisfy certain contraints, such as the row spacing in the lattice have to be larger than certain value (`Dmin`). 

In [ ]:
import numpy as np

from braket.ahs.atom_arrangement import AtomArrangement
from ahs_utils import show_register, show_global_drive, plot_avg_density_2D

from SSLattice import check_geometry_fit_constraints, CheckOmegaAndDelta, get_SS_lattice_coords_filter_atoms

def get_SS_lattice(
    Nx: int, 
    Ny: int, 
    V2_over_V1: float = 1.0, 
    Delta_over_V1: float = 2.0, 
    Dmin = 2.0e-6, 
    phase: str="1/3", 
    check_lattice_validity=True
):
    """
    Create a SS lattice for the specified phase
    
    Args:
        Nx (int): The number of atoms along the x direction (meters)
        Ny (int): The number of atoms along the y direction (meters)
        V2_over_V1 (float): The ratio of the vectors V2 and V1. Default is 1.0. (unitless)
        Delta_over_V1 (float): The ratio of final detuning with V1. Default is 2.0. (unitless)
        Dmin (float): The minimum row spacing. Default is 2.0e-6. (meters)
        phase (string): The phase realized on the SS lattice, either "1/3" or "2/5". Default is "1/3"
        check_lattice_validity (bool): If True, check if the lattice can be realized on the Aquila device. Default is True.
        
    Returns:
        register (AtomArrangement): The atom arrangement for the SS lattice
        filter_atoms (List[Int]): The indices for the boundary atoms
    """    
    
    assert phase in ["1/3", "2/5"]
    
    coords, filter_atoms = get_SS_lattice_coords_filter_atoms(Nx, Ny, V2_over_V1, Delta_over_V1, Dmin, phase)
    
    # An AtomArrangement object is realized by
    # putting in the coordinates of the atoms
    register = AtomArrangement()
    for coord in coords:
        register.add(coord)    
    
    # Check Geometry Constraints
    if check_lattice_validity:
        check_geometry_fit_constraints(register)    
    
    return register, filter_atoms
    

Below we show two example atom arrangments for the 2/5 and 1/3 phases respectively

In [ ]:
## Define the lattice for the 2/5 phase

Nx, Ny = 8, 9
V2_over_V1 = 1.0
Delta_over_V1 = 2.5
Dmin = 2.1e-6

register_25, filter_atoms_25 = get_SS_lattice(Nx, Ny, phase="2/5", V2_over_V1=V2_over_V1, Delta_over_V1=Delta_over_V1, Dmin=Dmin)
print(f"The indices for the boundary atoms of the 2/5 phase = {filter_atoms_25}")
show_register(register_25)


In [ ]:
## Define the lattice for the 1/3 phase

Nx, Ny = 8, 8
V2_over_V1 = 1.25
Delta_over_V1 = 1.5
Dmin = 2.18e-6

register_13, filter_atoms_13 = get_SS_lattice(Nx, Ny, phase="1/3", V2_over_V1=V2_over_V1, Delta_over_V1=Delta_over_V1, Dmin=Dmin)
print(f"The indices for the boundary atoms of the 1/3 phase = {filter_atoms_13}")
show_register(register_13)


### Driving field for SS lattice

Once the register for a SS lattice is defined, all the atoms are initialized to the ground states at the beginning of the quantum evolution. In order to realize the specified phase, we define a driving field that adiabatically evolve the system to the desired ground state, where certain atoms in the SS lattice are excited to the Rydberg states. Besides `V2_over_V1`, `Delta_over_V1` and `Dmin`, the parameters that determine the phase of interest, the driving field also depends on the parameter `Rb_over_R1`, which specifies the maximum Rabi frequency of the Hamiltonian. 


In [ ]:
from braket.timings.time_series import TimeSeries
from braket.ahs.driving_field import DrivingField

from SSLattice import get_drive_params

def get_drive(V2_over_V1: float = 1.1, Delta_over_V1: float = 2.0, Dmin = 2.08e-6, Rb_over_R1 = 1.1, check_drive_validity=True):
    """
    Create a time dependent Hamiltonian
    
    Args:
        V2_over_V1 (float): The ratio of the vectors V2 and V1. Default is 1.1 (unitless)
        Delta_over_V1 (float): The ratio of final detuning with V1. Default is 2.0 (unitless)
        Dmin (float): The minimum row spacing (meters)
        Rb_over_R1 (float): The parameter determines the maximum Rabi frequency and the Rydberg blockade radius (unitless)
        check_drive_validity (bool): If True, check if the drive can be realized on the Aquila device. Default is True.
        
    Returns:
        drive (DrivingField): The time dependent Hamiltonian
    """    

    Omega, Delta_start, Delta_end = get_drive_params(V2_over_V1, Delta_over_V1, Dmin, Rb_over_R1, check_drive_validity=check_drive_validity)
        
    
    # To define a DrivingField object, 
    # we specify its parameters,
    # amplitude, phase and detuing
    # as time-value pairs
    
    time_max = 4e-6
    time_ramp = 2.5e-07    
    
    delta = TimeSeries()
    delta.put(0.0, Delta_start)
    delta.put(time_ramp, Delta_start)
    delta.put(time_max - time_ramp, Delta_end)
    delta.put(time_max, Delta_end)    

    omega = TimeSeries()
    omega.put(0.0, 0.0)
    omega.put(time_ramp, Omega)
    omega.put(time_max - time_ramp, Omega)
    omega.put(time_max, 0.0)
    
    phi = TimeSeries().put(0.0, 0.0).put(time_max, 0.0)

    drive = DrivingField(amplitude=omega, phase=phi, detuning=delta)
    
    return drive

Below, we show one example of the driving field. Here `amplitude`, `detuning` and `phase` correspond to the time dependent parameters in $H(t)$ defined above.

In [ ]:
drive = get_drive()
show_global_drive(drive)

### Probe the phase transitions of SS lattices with tensor nework simulation using Braket hybrid jobs

With the register and driving field defined, we are ready to realize different phases of the SS lattice. In particular, we are interested in studying the phase transtion of the SS lattice, by varying $\Delta/V_1$ for both the 1/3 and 2/5 phases, followed by measuring the respective order parameters. Since each simulation, with different value of $\Delta/V_1$, is independent from the others, we can run these simulation tasks in parallel. In particular, we will use Amazon Braket Hybrid Jobs to parallelize the tasks on a large instance to speed up our simulation.

We first define the tasks for 1/3 and 2/5 phases with different values of $\Delta/V_1$.

In [ ]:
def get_programs_and_filter_atoms(
    phase:str, 
    params:list, 
    Delta_over_V1range: list, 
    check_lattice_validity: bool=True,
    check_drive_validity: bool=True
):
    """
    Create an AHS program for the given phase and parameters
    
    Args:
        phase (str): The phase of interest
        params (list): The parameters for the phase
        Delta_over_V1range (list): A list of varying Delta_over_V1 (the ratio between Delta and V1)
        check_lattice_validity (bool): If True, check if the lattice can be realized on the Aquila device. Default is True.
        check_drive_validity (bool): If True, check if the drive can be realized on the Aquila device. Default is True.
        
    Returns:
        program (AnalogHamiltonianSimulation): The AHS program for the phase
        filter_atoms (List[Int]): The indices for the boundary atoms
        
    Example:
        >>> params = (8, 8, 1.0, 2.0e-6)
        >>> phase = "1/3"
        >>> programs, filter_atoms = get_programs_and_filter_atoms(phase, params, [1.0])
    """    
        
        
    (Nx, Ny, V2_over_V1, Dmin) = params

    programs = []
    for Delta_over_V1 in Delta_over_V1range:

        # `filter_atoms` is the same for different values of Delta_over_V1
        register, filter_atoms = get_SS_lattice(
            Nx, Ny, 
            phase=phase, 
            V2_over_V1=V2_over_V1, 
            Delta_over_V1=Delta_over_V1, 
            Dmin=Dmin, 
            check_lattice_validity=check_lattice_validity
        )
        drive = get_drive(
            V2_over_V1=V2_over_V1, 
            Delta_over_V1=Delta_over_V1, 
            Dmin=Dmin, 
            check_drive_validity=check_drive_validity
        )
        program = AnalogHamiltonianSimulation(register=register, hamiltonian=drive)
        programs.append(program)
    
    return programs, filter_atoms

In [ ]:
Delta_over_V1range = np.linspace(0, 4, 10)

# Define the programs for the 1/3 phase
Nx, Ny = 8, 8
V2_over_V1_13 = 1.25
Dmin = 2.18e-6

params_13 = (Nx, Ny, V2_over_V1_13, Dmin)
programs_13, filter_atoms_13 = get_programs_and_filter_atoms("1/3", params_13, Delta_over_V1range, False, False) ;

# Define the programs for the 2/5 phase
Nx, Ny = 8, 9
V2_over_V1_25 = 1.0
Dmin = 2.00e-6

params_25 = (Nx, Ny, V2_over_V1_25, Dmin)
programs_25, filter_atoms_25 = get_programs_and_filter_atoms("2/5", params_25, Delta_over_V1range, False, False) ;

Next we define the function `get_job_phase_transition` which parallelizes the simulation for a given phase (either 1/3 or 2/5 phase) on the instance `"ml.c4.8xlarge"`. Please refer to [this page](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-configure-job-instance-for-script.html) for detailed descriptions of all the instance available.

In [ ]:
from braket.jobs import hybrid_job
from braket.devices import LocalSimulator
from braket.jobs.config import InstanceConfig
import multiprocessing as mp

import boto3
account_id = boto3.client("sts").get_caller_identity()["Account"]

image_uri = f"{account_id}.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-qcup:latest"
instance_type = "ml.c4.8xlarge"
instance_config=InstanceConfig(instanceType=instance_type)

@hybrid_job(device="", image_uri=image_uri, instance_config=instance_config)
def get_job_phase_transition(programs):

    tn_simulator = LocalSimulator('braket_ahs_tn')
    
    with mp.Pool(min(mp.cpu_count(), len(programs))) as pool:
        tasks = pool.map(tn_simulator.run, programs)
        
    return [task.result() for task in tasks]

<div class="alert alert-block alert-info">
The message will appear when we are using custom image instead of the default image for Jobs. Here we have make sure that the python version in the local environment matches that of the container.
</div>


We proceed to submit two jobs, for the 1/3 and 2/5 phases, respectively.

In [ ]:
job_13 = get_job_phase_transition(programs_13)
job_25 = get_job_phase_transition(programs_25)

For convience, we print out the ARN and the status of the jobs

In [ ]:
print("For the 2/5 phase")
print(f"ARN: {job_25.arn}")
print(f"status: {job_25.state()}")

print("For the 1/3 phase")
print(f"ARN: {job_13.arn}")
print(f"status: {job_13.state()}")

Suppose we have lost the connection to the notebook, the job can always be recovered from its ARN. For example,
```
from braket.aws.aws_quantum_job import AwsQuantumJob
job_13 = AwsQuantumJob(arn="arn:aws:braket:us-east-1:000000000000:job/xxxxxxx")
job_25 = AwsQuantumJob(arn="arn:aws:braket:us-east-1:000000000000:job/yyyyyyy")
```

Once the jobs are completed, we can retrieve their results.

In [ ]:
%%time

results_13 = job_13.result()['result']
results_25 = job_25.result()['result']

We will visualize the phase transition by plotting the change of the average densities, and the order parameters, as a function of $\Delta/V_1$.

In [ ]:
import matplotlib.pyplot as plt
from SSLattice import analyze

def visualize_phase_transition(V2_over_V1, results, params, programs, filter_atoms, Delta_over_V1range=Delta_over_V1range):
    (Nx, Ny, _, _) = params
    order_params_13, order_params_25 = [], []
    for ind, (result, program) in enumerate(zip(results, programs)):
        
        # Here, `analyze` calculates the 1/3 and 2/5 order parameters
        # and plot the average Rydberg density and the structure factors
        Delta_over_V1 = Delta_over_V1range[ind]
        order_param_13, order_param_25 = analyze(Nx, Ny, result, program, filter_atoms, Delta_over_V1)
        order_params_13.append(order_param_13)
        order_params_25.append(order_param_25)
        
    plt.plot(Delta_over_V1range, order_params_13, label="1/3 order parameter")
    plt.plot(Delta_over_V1range, order_params_25, label="2/5 order parameter")
    plt.xlabel(r'$\Delta/V_1$')
    plt.ylabel('Order parameters')
    plt.title(f'Order parameters at V2/V1 = {V2_over_V1}')
    plt.legend()

In [ ]:
visualize_phase_transition(V2_over_V1_13, results_13, params_13, programs_13, filter_atoms_13)

We observe that the signatures of the 1/3 phase, including the peaks in the Fourier transform and 1/3 order parameters, become prominent at around $\Delta/V_1=1.0$, and drop quickly after $\Delta/V_1=2.5$. The 2/5 order surpasses the 1/3 order at around $\Delta/V_1=3.2$ which signifies a phase transition from the 1/3 to 2/5 phase. Recall we have fixed ratio $V_2/V_1=1.25$ in this set of simulation, the observed phase transition agrees qualitatively from the phase diagram, as shown below.

<center><img src="phases_SS_lattice_13.png" alt="drawing" style="width:350px;"/>

In [ ]:
visualize_phase_transition(V2_over_V1_25, results_25, params_25, programs_25, filter_atoms_25)

Similarly, we observe that the signatures of the 2/5 phase, including the peaks in the Fourier transform and 2/5 order parameters, become prominent at around $\Delta/V_1=1.5$, and drop slightly after $\Delta/V_1=3.5$. Recall we have fixed ratio $V_2/V_1=1.0$ in this set of simulation, the observed phase transition agrees qualitatively from the phase diagram, as shown below.

<center><img src="phases_SS_lattice_25.png" alt="drawing" style="width:350px;"/>

<div class="alert alert-block alert-info">
Exercise 2: Adjust the parameters `V2_over_V1` and rerun the jobs for different phases
    
Exercise 3: Adjust the parameters in the driving field, such as its duration, and rerun the jobs for different phases
</div>


## Simulation of Shastry-Sutherland lattice with QuEra's Aquila device

In previous sections, we have used classical simulators to simulate the phases of the SS lattice. Here we switch gear and show how to simulate these phases on QuEra's Aquila device. We could use the same AHS programs defined above, and simply replace the backend from classical simulator to the QPU. However, in order to showcase the full capabilities of the Aquila device, we will realize the 1/3 and 2/5 phases on bigger lattices.

In [ ]:
Nx, Ny = 8, 14

## Define the lattice for the 2/5 phase on taller lattice of size 8x14
V2_over_V1 = 0.82
Delta_over_V1_25 = 2.5
Dmin = 2.00001e-6
params_25 = (Nx, Ny, V2_over_V1, Dmin)

programs_25, filter_atoms_25 = get_programs_and_filter_atoms("2/5", params_25, [Delta_over_V1_25]) ;
program_25 = programs_25[0]

## Define the lattice for the 1/3 phase on taller lattice of size 8x14
V2_over_V1 = 1.25
Delta_over_V1_13 = 1.5
Dmin = 2.18e-6
params_13 = (Nx, Ny, V2_over_V1, Dmin)

programs_13, filter_atoms_13 = get_programs_and_filter_atoms("1/3", params_13, [Delta_over_V1_13]) ;
program_13 = programs_13[0]

<div class="alert alert-block alert-info">
Here we have used the experimental capabilities of the Aquila device, namely the "Tall geometries" and "Tight geometries", which provides more flexibilities for defining atomic arrays. In order to get access to these experimental capabilities, one will need to explicitly enable them through the Amazon Braket console.
</div>


Below we show how to use Hybrid Jobs to run tasks on the Aquila device. We will run the two programs on Aquila device with 100 shots each. Note that, in order to satisfy the resolution requirements of the device, we will need to discretize the program before the submission.

In [ ]:
from braket.aws import AwsDevice
from braket.devices import Devices

device_arn = Devices.QuEra.Aquila

@hybrid_job(device=device_arn)
def get_job_13_25_phases(program_13, program_25, shots):
    qpu = AwsDevice(device_arn)

    qpu_task_25 = qpu.run(program_25.discretize(qpu), shots=shots)
    qpu_task_13 = qpu.run(program_13.discretize(qpu), shots=shots)
    
    result_qpu_25 = qpu_task_25.result()
    result_qpu_13 = qpu_task_13.result()
    
    return result_qpu_13, result_qpu_25


In [ ]:
shots = 100
job_aquila = get_job_13_25_phases(program_13, program_25, shots)

print("For the job running on Aquila")
print(f"ARN: {job_aquila.arn}")
print(f"status: {job_aquila.state()}")

Again, suppose we have lost the connection to the notebook, the job can always be recovered from its ARN as
```
from braket.aws.aws_quantum_job import AwsQuantumJob
job_aquila = AwsQuantumJob(arn="arn:aws:braket:us-east-1:000000000000:job/xxxxxxx")
```

The runtime for the above job depends on the length of the queue for the Aquila device. Once the job is completed, we can retrieve its result as follows.

In [ ]:
result_qpu_13, result_qpu_25 = job_aquila.result()['result']

In [ ]:
order_param_13, order_param_25 = analyze(Nx, Ny, result_qpu_25, program_25, filter_atoms_25, Delta_over_V1_25)
print(f"order_param_13 = {order_param_13}\norder_param_25 = {order_param_25}")

In [ ]:
order_param_13, order_param_25 = analyze(Nx, Ny, result_qpu_13, program_13, filter_atoms_13, Delta_over_V1_13)
print(f"order_param_13 = {order_param_13}\norder_param_25 = {order_param_25}")

From the above figures, we can see the signatures of both the 2/5 and 1/3 phases for the SS lattice, realized on the Aquila device.

<div class="alert alert-block alert-info">
Exercise 4: Submit a job <em>without</em> discretizing it with respect to the QPU. What did you see and why?
    
Exercise 5: Vary the parameters and resubmit the jobs
</div>


## Summary
In this notebook, we have demonstrated several methods to simulate the phases of the SS lattice on Amazon Braket, including using local simulator, hybrid jobs and QuEra's Aquila device. For the interested readers, they are encouraged to check out [more AHS examples](https://github.com/amazon-braket/amazon-braket-examples/tree/main/examples/analog_hamiltonian_simulation) in the [amazon-braket-examples repository](https://github.com/amazon-braket/amazon-braket-examples), and research highlightes in [AWS Quantum Technologies Blog](https://aws.amazon.com/blogs/quantum-computing/).

<div class="alert alert-block alert-info"> 
<b>You have access to this environment for the rest of the week. We encourage you to use it to explore Amazon Braket at your own pace. We are here for the rest of the day to help you. Please don't hesitate to approach us.</b>
</div>